In [2]:
import pandas as pd
d = {"headers": {"Employee": ["id", "company", "salary"]}, "rows": {"Employee": [[1, "A", 2341],[2, "A", 341],[3, "A", 15],[4, "A", 15314],[5, "A", 451],[6, "A", 513],[7, "B", 15],[8, "B", 13],[9, "B", 1154],[10, "B", 1345],[11, "B", 1221],[12, "B", 234],[13, "C", 2345],[14, "C", 2645],[15, "C", 2645],[16, "C", 2652],[17, "C", 65]]}}
pd.DataFrame(d['rows']['Employee'], columns = d['headers']['Employee']).to_csv("./Employee.txt", index=None)

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("median employee salary").config("pyspark.sql.shuffle.partition", "4").getOrCreate()

In [21]:
e = spark.read.option("header", True).csv("./Employee.txt")
e.show()
e = e.withColumn("salary", e.salary.cast('int'))
e = e.withColumn("id", e.id.cast('int'))
e.printSchema()

+---+-------+------+
| id|company|salary|
+---+-------+------+
|  1|      A|  2341|
|  2|      A|   341|
|  3|      A|    15|
|  4|      A| 15314|
|  5|      A|   451|
|  6|      A|   513|
|  7|      B|    15|
|  8|      B|    13|
|  9|      B|  1154|
| 10|      B|  1345|
| 11|      B|  1221|
| 12|      B|   234|
| 13|      C|  2345|
| 14|      C|  2645|
| 15|      C|  2645|
| 16|      C|  2652|
| 17|      C|    65|
+---+-------+------+

root
 |-- id: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- salary: integer (nullable = true)



Write an SQL query to find the median salary of each company.

Return the result table in any order.

The query result format is in the following example.

In [22]:
from pyspark.sql.functions import *
median = e.groupBy("company").count().withColumnRenamed('count', 'cnt').select('company', round(col('cnt')/2).alias("fi"), round((col('cnt')+1)/2).alias("se"))
median.show()
from pyspark.sql.window import Window

Window_spec = Window.partitionBy('company').orderBy('salary')
e.select('*', row_number().over(Window_spec).alias("row")).show()
e.select('*', row_number().over(Window_spec).alias("row")).join(median,'company').where(col('row').between(col('fi'), col('se'))).show()

+-------+---+---+
|company| fi| se|
+-------+---+---+
|      B|3.0|4.0|
|      C|3.0|3.0|
|      A|3.0|4.0|
+-------+---+---+

+---+-------+------+---+
| id|company|salary|row|
+---+-------+------+---+
|  8|      B|    13|  1|
|  7|      B|    15|  2|
| 12|      B|   234|  3|
|  9|      B|  1154|  4|
| 11|      B|  1221|  5|
| 10|      B|  1345|  6|
| 17|      C|    65|  1|
| 13|      C|  2345|  2|
| 14|      C|  2645|  3|
| 15|      C|  2645|  4|
| 16|      C|  2652|  5|
|  3|      A|    15|  1|
|  2|      A|   341|  2|
|  5|      A|   451|  3|
|  6|      A|   513|  4|
|  1|      A|  2341|  5|
|  4|      A| 15314|  6|
+---+-------+------+---+

+-------+---+------+---+---+---+
|company| id|salary|row| fi| se|
+-------+---+------+---+---+---+
|      B|  9|  1154|  4|3.0|4.0|
|      B| 12|   234|  3|3.0|4.0|
|      C| 14|  2645|  3|3.0|3.0|
|      A|  6|   513|  4|3.0|4.0|
|      A|  5|   451|  3|3.0|4.0|
+-------+---+------+---+---+---+

